## PII Text Redaction with Azure Language Service and Foundry Agents

![lab_flow](./Assets/lab_flow.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity azure-ai-textanalytics==5.3.0

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
azure_language_endpoint = os.getenv("AZURE_LANGUAGE_ENDPOINT")
azure_language_api_key = os.getenv("AZURE_LANGUAGE_API_KEY")

### Setting up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating an OpenAI Client

In [ ]:
# creating an openai client first
openai_client = project_client.get_openai_client()

### Create a Text Analytics Client

In [ ]:
ta_credential = AzureKeyCredential(azure_language_api_key)

text_analytics_client = TextAnalyticsClient(
    endpoint=azure_language_endpoint,
    credential=ta_credential
)

### Creating a Function to Perform Text Redaction

In [ ]:
def PII_Text_Redaction(input_text: str) -> str:
    documents = [
        input_text
    ]

    response = text_analytics_client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        redacted_text = doc.redacted_text
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("	Category: {}".format(entity.category))
            print("	Confidence Score: {}".format(entity.confidence_score))
            print("	Offset: {}".format(entity.offset))
            print("	Length: {}".format(entity.length))
        return redacted_text

### Setting Up Our Agent

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

agent_name = "travel-assistant-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a travel assistant. Help users plan their trips, find flights, hotels, and provide travel advice.",
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Conversation Object for Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat With the Agent

In [ ]:
chat = True

while chat:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        chat = False
        print("Exiting chat. Goodbye!")
    else:
        redacted_input = PII_Text_Redaction(user_input)

        print(f"Redacted Input: {redacted_input}")
        
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = redacted_input
        )

        print(f"Agent: {response.output_text}")